# CS-E-106: Data Modeling
## Fall 2019: HW 03

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import statsmodels.api as sm
from statsmodels.formula.api import ols
import seaborn as sns
from scipy import stats
from sklearn.model_selection import train_test_split

**Solution 1:**

In [116]:
def reg_loop(df, x_cols, y_col):
    r2_list = []
    lm_fits = {}
    for i in range(len(x_cols)):
        x_str = x_cols[i]
        y_str = y_col[0]
        x_str_c = str("C(%s)"%x_str) if df[x_cols[i]].dtype=="O" else x_str
        formula = y_str+" ~ "+x_str_c
        lm = ols(formula, data=df).fit()
        r2_list.append(lm.rsquared)
        lm_fits[x_cols[i]] = lm
    
    r2_df = pd.DataFrame((x_cols, r2_list), index=["Variable", "R-squared"]).T
    r2_df = r2_df.sort_values(by="R-squared", ascending=False)
    
    return(r2_df)

# def reg_loop(df, x_cols, y_col):
#     r2_list = []
#     for i in range(len(x_cols)):
#         X = sm.add_constant(df[x_cols[i]])
#         X = X.astype(float) if df[x_cols[i]].dtype != "O" else X
#         lm = sm.OLS(df[y_col].astype(float), X).fit()
#         r2_list.append(lm.rsquared)
    
#     r2_df = pd.DataFrame((x_cols, r2_list), index=["Variable", "R-squared"]).T
#     r2_df = r2_df.sort_values(by="R-squared", ascending=False)
    
#     return(r2_df)

In [126]:
cdi = pd.read_csv("data/CDI.csv")
mapper = {"Identification number": "ID", 
          "County": "county", "State": "state",
          'Land area':"area",
          'Total population':"population", 
          'Percent of population aged 18-34':"age18_34",
          'Percent of population 65 or older': "above65", 
          'Number of active physicians': "physicians",
          'Number of hospital beds':"hospital_beds", 
          'Total serious crimes':"serious_crimes",
          'Percent high school graduates': "highschool_grads", 
          "Percent bachelor's degrees": "bachelors_degree",
          'Percent below poverty level':"poverty_level", 
          'Percent unemployment':"unemployment_rate",
          'Per capita income': "per_capita_income", 
          'Total personal income': "personal_income", 
          'Geographic region': "region"}
cdi = cdi.rename(columns=mapper)

In [127]:
exc_cols = ["ID","physicians"]
x_cols = list(set(mapper.values())^set(exc_cols))
y_col = ["physicians"]
r2_df = reg_loop(df=cdi, x_cols=x_cols, y_col=y_col)
r2_df

,Variable,R-squared
14,county,0.921237
9,hospital_beds,0.903383
10,personal_income,0.898914
0,population,0.884067
3,serious_crimes,0.673154
7,per_capita_income,0.0999411
1,state,0.0634583
8,bachelors_degree,0.0560579
2,age18_34,0.0143279
5,area,0.00609565


Thus, **county accounts for maximum variability in the number of active physicians**. The remainder variables and their respective $R^{2}$ is given below in descending order of importance.

**Solution 2:**

In [145]:
def confint_regions(df):
    regions = df['region'].unique()
    lm_fits = {}
    for i in regions:
        formula = "hospital_beds ~ bachelors_degree" 
        lm = ols(formula, data=df[df.region==i]).fit()
        lm_fits[i] = lm
        print("\n Region: %s"%i)
        print("\n Confidence Interval:")
        print(lm.conf_int(alpha=0.1))
        print("\n Coefficients:")
        print(lm.params)

In [146]:
confint_regions(df=cdi)


 Region: 4

 Confidence Interval:
                            0            1
Intercept        -1531.492253  2543.831299
bachelors_degree   -39.345730   136.372057

 Coefficients:
Intercept           506.169523
bachelors_degree     48.513164
dtype: float64

 Region: 2

 Confidence Interval:
                           0            1
Intercept        -293.397089  2008.633955
bachelors_degree  -23.139420    85.847861

 Coefficients:
Intercept           857.618433
bachelors_degree     31.354221
dtype: float64

 Region: 3

 Confidence Interval:
                           0            1
Intercept         259.279863  1523.562478
bachelors_degree   -6.102009    49.901786

 Coefficients:
Intercept           891.421170
bachelors_degree     21.899888
dtype: float64

 Region: 1

 Confidence Interval:
                          0            1
Intercept        -80.189409  1788.001195
bachelors_degree -10.690032    70.750370

 Coefficients:
Intercept           853.905893
bachelors_degree     30.030169

**Solution 3:**

**(a)**

In [148]:
gpa = pd.read_csv("data/GPA.csv")
lm_gpa = ols(formula="GPA~ACT", data=gpa).fit()
lm_gpa.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    GPA   R-squared:                       0.073
Model:                            OLS   Adj. R-squared:                  0.065
Method:                 Least Squares   F-statistic:                     9.240
Date:                Tue, 08 Oct 2019   Prob (F-statistic):            0.00292
Time:                        15:36:01   Log-Likelihood:                -112.50
No. Observations:                 120   AIC:                             229.0
Df Residuals:                     118   BIC:                             234.6
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      2.1140      0.321      6.588      0.000       1.479       2.750
ACT            0.0388      0.013      3.040      0.003       0.014       0.064
==============================================================================
Omnibus:                       26.969   Durbin-Watson:                   1.831
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               47.360
Skew:                          -0.994   Prob(JB):                     5.20e-11
Kurtosis:                       5.349   Cond. No.                         142.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [149]:
sm.stats.anova_lm(lm_gpa)

,df,sum_sq,mean_sq,F,PR(>F)
ACT,1.0,3.587846,3.587846,9.240243,0.002917
Residual,118.0,45.817608,0.388285,NaN,NaN


**(b)**

$MSR = \sum_{i=1}^n({\hat{Y_{i}}-\bar{Y})}^{2}$

MSR measures the effect of the regression line in explaining the total variation in $Y_{i}$.

$MSE = \frac{\sum_{i=1}^n(Y_{i}-\hat{Y_{i}})^{2}}{n-2}$

MSE measures the mean variation of $Y_{i}$ around the regresion line. Its the average of all the squared distances by which the regression line missed the actual $Y_{i}$.

$E[MSE] = \sigma^{2}$

$E[MSR] = \sigma^{2}+\beta_{1}\sum_{i=1}^n{(X_{i}-\bar{X})^{2}}$

Thus, MSE and MSR will estimate same quantity when $\beta_{1}=0$ i.e. $Y_{i} = \bar{Y}$

**(c)**

*Null Hypothesis:* $H_{0}: \beta_{1}=0$;
*Alternate Hypothesis:* $H_{1}: \beta_{1}\neq0$

*Decision Rule:* 

$F^{*} = \frac{MSR}{MSE}$

- If $F* \leq F(1-\alpha; 1, n-2)$, conclude $H_{0}$;

- If $F* \geq F(1-\alpha; 1, n-2)$, conclude $H_{1}$

In [150]:
MSR = 3.5878
MSE = 0.3883 
F = MSR/MSE
print(F)

9.2397630697914


In [162]:
stats.f.cdf(0.01,1,118)

0.07948597539577211

*Result:*

Thus, since $F^{*} > F(1-\alpha; 1, n-2)$, we conclude that $H_{1}: \beta_{1}\neq0$ holds.

**(d)**

The absolute magnitude of the reduction in the variation of Y when X is introduced into the regression model is SST - SSE = SSR = 3.588 (from the ANOVA table above).

The relative measure is given by $\frac{SSR}{SST} = \frac{3.588}{3.588+45.818} = 0.0726$. This measure is also known as the $R^{2}$ or the coefficient of determination.

In [163]:
R_sq = 3.588/(3.588+45.818)
R_sq

0.07262275836942882

**(e)**

In [165]:
r = np.sqrt(R_sq)
r

0.2694861005124918

Looking at the summary of the regression model for GPA dataset, $\beta_{1}$ is positive. Hence, $r = +0.27$.

**(f)**
 
Operationally, $R^{2}$ has more clear interpretation.

- $R^{2}$ is the proportion of total variation in Y explained by X. Thus, it is a relative measure of improvement that was made by the introduction of X in the regression model. This can be used in a more direct way compared to r which measures linear association between X and Y.

- $R^{2}$ is on a scale of 0 to 1 (1 indicating the highest correlation), whereas, r ranges from -1 to 1 (the extremes indicating highest correlation). Meaning both r=-1 and r=+1 can mean the same level of association between X and Y. Also, the objective of the coefficients of correlation/determination is to measure the overall effectiveness of the model rather than looking at which direction the regression line is going. This is better accomplished by $R^{2}$.

**Solution 4:**

**(a)**

In [175]:
crime = pd.read_csv("data/Crime Rate.csv")

In [176]:
stats.pearsonr(crime["Y"], crime["X"])

(-0.41270331301195956, 9.571395791542376e-05)

**(b)**

In [170]:
lm_crime = ols(formula="Y~X", data=crime).fit()
lm_crime.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      Y   R-squared:                       0.170
Model:                            OLS   Adj. R-squared:                  0.160
Method:                 Least Squares   F-statistic:                     16.83
Date:                Tue, 08 Oct 2019   Prob (F-statistic):           9.57e-05
Time:                        16:31:52   Log-Likelihood:                -770.43
No. Observations:                  84   AIC:                             1545.
Df Residuals:                      82   BIC:                             1550.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   2.052e+04   3277.643      6.260      0.000     1.4e+04     2.7e+04
X           -170.5752     41.574     -4.103      0.000    -253.280     -87.871
==============================================================================
Omnibus:                        2.224   Durbin-Watson:                   1.495
Prob(Omnibus):                  0.329   Jarque-Bera (JB):                2.229
Skew:                           0.360   Prob(JB):                        0.328
Kurtosis:                       2.655   Cond. No.                     1.01e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.01e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [171]:
sm.stats.anova_lm(lm_crime)

,df,sum_sq,mean_sq,F,PR(>F)
X,1.0,9.346294e+07,9.346294e+07,16.833765,0.000096
Residual,82.0,4.552732e+08,5.552112e+06,NaN,NaN


*Null Hypothesis:* $H_{0}: \beta_{1}=0$;
*Alternate Hypothesis:* $H_{1}: \beta_{1}\neq0$

*Decision Rule:* 

$F^{*} = \frac{MSR}{MSE}$

- If $F* \leq F(1-\alpha; 1, n-2)$, conclude $H_{0}$;

- If $F* \geq F(1-\alpha; 1, n-2)$, conclude $H_{1}$

In [172]:
MSR = 93462942
MSE = 5552112 
F = MSR/MSE
print(F)

16.833763800153886


In [173]:
stats.f.cdf(0.01,1,82)

0.07941159069686124

*Result:*

Thus, since $F^{*} > F(1-\alpha; 1, n-2)$, we conclude that $H_{1}: \beta_{1}\neq0$ holds.

**(c)**

In [174]:
stats.spearmanr(crime["Y"], crime["X"])

SpearmanrResult(correlation=-0.4259324213180894, pvalue=5.3585991604908214e-05)

**(d)**
 
*Null Hypothesis:* There is no association between X and Y;
*Alternate Hypothesis:* There is an association between X and Y

*Decision Rule:* 

$t^{*} = \frac{r_{s}\sqrt{n-2}}{1-r_{s}^{2}}$

- If $|{t^{*}}| \leq t(1-\alpha/2; n-2)$, conclude $H_{0}$;

- If $|{t^{*}}| \geq t(1-\alpha/2; n-2)$, conclude $H_{1}$

In [179]:
r_s = -0.4259324
n = crime.shape[0]
t = (r_s*np.sqrt(n-2)/(1-r_s**2))
t

-4.711786789441044

In [181]:
stats.t.cdf(0.005,82)

0.5019886309891158

*Result:*

Thus, since $|{t^{*}}| \geq t(1-\alpha/2; n-2)$, we conclude that $H_{1}$ that there is an association between X and Y.
